# Processing the Raw Data to Get Partisanship Data

This notebook takes the raw CSV vote data and compiles it into the number about party unity and party agreement

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
%matplotlib inline
sns.set(style="ticks")

leg = pd.read_csv('legislators.csv')

import matplotlib.pyplot as plt

/home/anne/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/home/anne/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,20,21,22,23,24,26,27,28,30,31,32,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
'''
This function takes a session of congress 1-113

It reads the csvs and returns:
- a list of parties present in that session
- a list of objects. each object represents a vote in that session of congress, and each key of the object is a
    party. the values are a list of length 2. the first number in the list is the number of people in that
    party who voted Yea on that bill, and the second is people in the party who voted Nay.  it ignores non-voters
'''
def get_votes(sess):
    house = pd.read_csv('cleanedcsv2/'+str(sess)+'house.csv')
    senate = pd.read_csv('cleanedcsv2/'+str(sess)+'senate.csv')
    votes= []
    parties_init = {}
    includeAmendments = True

    # get parties in a session - HOUSE
    for col in house.columns:
        if col == "date":
            break
        party = leg.party[leg['vote_id']==col].iloc[0]
        if party not in parties_init:
            parties_init[party] = 1
        else:
            parties_init[party] +=1

    # get parties in a session - SENATE
    for col in senate.columns:
        if col == "date":
            break
        party = leg.party[leg['vote_id']==col].iloc[0]
        if party not in parties_init:
            parties_init[party] = 1
        else:
            parties_init[party] +=1
    print parties_init
    tot = sum(parties_init.values())

    parties = []

    for key in parties_init.keys():
        if float(parties_init[key])/tot > .05 and type(key) == type('str'):
            parties.append(key)

    print parties

    # iter house
    for index, row in house.iterrows():
        if not includeAmendments and row[isAmendment]:
            continue

        votedict = {}
        for party in parties:
            votedict[party] = [0,0] # count of [Yea, Nay]
        votedict['overall'] = [0,0]

        for col in row.keys():
            if col=="date":
                break
            party = leg.party[leg['vote_id']==col].iloc[0]
            if party in parties:
                vote = row[col]
                if vote == "Yea":
                    votedict[party][0] +=1
                    votedict['overall'][0] += 1
                elif vote == "Nay":
                    votedict[party][1] +=1
                    votedict['overall'][1] += 1
        votes.append(votedict)

    # iter senate
    for index, row in senate.iterrows():
        if not includeAmendments and row[isAmendment]:
            continue

        votedict = {}
        for party in parties:
            votedict[party] = [0,0] # count of [Yea, Nay]
        votedict['overall'] = [0,0]

        for col in row.keys():
            if col=="date":
                break
            party = leg.party[leg['vote_id']==col].iloc[0]
            if party in parties:
                vote = row[col]
                if vote == "Yea":
                    votedict[party][0] +=1
                    votedict['overall'][0] += 1
                elif vote == "Nay":
                    votedict[party][1] +=1
                    votedict['overall'][1] += 1
        votes.append(votedict)
    return votes, parties
            

In [41]:
'''
This makes a list representing how an "average party member" votes in that session

This function takes the votes list for a session and the list of parties. 
it returns a dictionary where the keys are parties and the values are a list (of length number of votes)
where each element is a 1 or a 0 representing whether the majority of the party voted Yea or Nay on that vote
'''
def get_avg_party(votes, parties):
    avg_party = {}

    for party in parties:
        avg_party[party] = []

    for res in votes:
        for party in parties:
            if res[party][0] > res[party][1]:
                avg_party[party].append(1)
            else:
                avg_party[party].append(0)

    return avg_party

In [52]:
'''
This function takes the dictionary of how the average party member votes and the number of votes in the session

it returns a dictionary, the important (and only used value of) is the "majorty" key
it's the count of votes where the majority of average party members vote the same way
'''
def get_party_agreement(avg_party, numvotes):
    party_agreement = {}

    for key in avg_party.keys():
        party_agreement[key] = 0
    party_agreement["majority"] = 0

    for i in range(numvotes):
        votes = {}
        for key in avg_party.keys():
            votes[key] = avg_party[key][i]

        # MAJORITY of parties all agree
        if sum(votes.values()) > len(votes)/2:
            party_agreement['majority'] += 1
        else:
            for party in votes.keys():
                if votes[party] == 1:
                    party_agreement[party] +=1

    return party_agreement

In [6]:
# round to 2 decimal places
def round2(fl):
    return int(fl*100)/100.0


In [133]:
'''
This runs the above functions for all sessions and converts results to percentages
'''

res = {}

for i in range(1,114):
    print i
    votes_dict, parties = get_votes(i)
    avg_party_dict = get_avg_party(votes_dict, parties)
    party_agreement_dict = get_party_agreement(avg_party_dict, len(votes_dict))
    
    print party_agreement_dict
    
    for key in res.keys():
        if key not in party_agreement_dict.keys():
            res[key].append(round2(float(0)))
    
    for key in party_agreement_dict.keys():
        if key in res:
            res[key].append(round2(float(party_agreement_dict[key])/len(votes_dict)))
        else:
            res[key] = [round2(float(0))]*(i-1)
            res[key].append(round2(float(party_agreement_dict[key])/len(votes_dict)))
            
    print res

81
{'Republican': 222, 'Democrat': 331, 'American Labor': 1}
['Republican', 'Democrat']
{'majority': 172, 'Republican': 256, 'Democrat': 182}
{'majority': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23], 'Republican': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35], 'Democrat': [0.

/home/anne/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (4,21,57,79,134,175,213,238,267,351,391,399) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/anne/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (4,6,22,67,145,162,173,221,239,241,308,314,324) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):



{'majority': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23, 0.24, 0.33, 0.38, 0.38, 0.4, 0.37, 0.27, 0.38, 0.48, 0.51, 0.46, 0.48], 'Republican': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35, 0.43, 0.2, 0.22, 0.24, 0.2, 0.27, 0.25, 0.25, 0.19, 0.15, 0.18, 0.18], 'Democrat': [0.

/home/anne/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/anne/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (1,5,21,33,66,155,169,216,220,232,266,305,395,434) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):



{'majority': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23, 0.24, 0.33, 0.38, 0.38, 0.4, 0.37, 0.27, 0.38, 0.48, 0.51, 0.46, 0.48, 0.45], 'Republican': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35, 0.43, 0.2, 0.22, 0.24, 0.2, 0.27, 0.25, 0.25, 0.19, 0.15, 0.18, 0.18, 0.18], 'De

...

/home/anne/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (10,41,55,59,62,94,95,96,112,133,134,148,150,151,154,173,210,231,253,264,266,304,305,317,360,402,403,417,433,449) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/anne/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (44,101) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):



{'majority': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23, 0.24, 0.33, 0.38, 0.38, 0.4, 0.37, 0.27, 0.38, 0.48, 0.51, 0.46, 0.48, 0.45, 0.48, 0.44, 0.46, 0.41, 0.36, 0.44, 0.44, 0.38, 0.34, 0.27, 0.34, 0.38, 0.37, 0.3, 0.27], 'Republican': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

/home/anne/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (0,41,60,98,121,122,132,162,167,256,264,295,302,317,332,362,385,403,407,409,433) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/anne/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (9,19,52,53,98,120,157,171,183,202,247,272,292,307,327,413,425,426) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):



{'majority': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23, 0.24, 0.33, 0.38, 0.38, 0.4, 0.37, 0.27, 0.38, 0.48, 0.51, 0.46, 0.48, 0.45, 0.48, 0.44, 0.46, 0.41, 0.36, 0.44, 0.44, 0.38, 0.34, 0.27, 0.34, 0.38, 0.37, 0.3, 0.27, 0.22, 0.22], 'Republican': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

/home/anne/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (1,2,5,11,17,45,91,114,247,296,351,417,438) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [7]:
'''
This calculates party unification.

It builds a dictionary - keys are parties, values are lists that get added to for each session
they represent the party unity metric
'''

party_partisanship = {}

for i in range(1,114):
    print i
    votes_dict, parties = get_votes(i)
    party_unification = {}
    
    for party in parties:
        party_unification[party]=0

    for vote in votes_dict:
        for party in parties:
            votelist = vote[party]
            # 90% majority on a vote means party is "unified" = partisanship point
            if (sum(votelist)>0):
                if float(max(votelist))/sum(votelist) >= 0.9:
                    party_unification[party] += 1
                
                
    for key in party_partisanship.keys():
        if key not in party_unification.keys():
            party_partisanship[key].append(round2(float(0)))
    
    for key in party_unification.keys():
        if key in party_partisanship:
            party_partisanship[key].append(round2(float(party_unification[key])/len(votes_dict)))
        else:
            party_partisanship[key] = [round2(float(0))]*(i-1)
            party_partisanship[key].append(round2(float(party_unification[key])/len(votes_dict)))
    
    print party_partisanship
    
            

1
{nan: 34, 'Federalist': 24, 'Anti-Administration': 6, 'Pro-Administration': 11, 'Republican': 19, 'Democratic Republican': 1}
['Federalist', 'Anti-Administration', 'Pro-Administration', 'Republican']
{'Federalist': [0.06], 'Anti-Administration': [0.17], 'Republican': [0.27], 'Pro-Administration': [0.54]}
2
{nan: 33, 'Federalist': 30, 'Anti-Administration': 6, 'Pro-Administration': 6, 'Jackson': 1, 'Republican': 26}
['Federalist', 'Anti-Administration', 'Pro-Administration', 'Republican']
{'Federalist': [0.06, 0.1], 'Anti-Administration': [0.17, 0.03], 'Republican': [0.27, 0.14], 'Pro-Administration': [0.54, 0.07]}
3
{nan: 36, 'Federalist': 44, 'Jackson': 2, 'Pro-Administration': 4, 'Anti-Administration': 4, 'Republican': 51, 'Democratic Republican': 1}
['Federalist', 'Republican']
{'Federalist': [0.06, 0.1, 0.31], 'Anti-Administration': [0.17, 0.03, 0.0], 'Republican': [0.27, 0.14, 0.26], 'Pro-Administration': [0.54, 0.07, 0.0]}
4
{'Anti-Administration': 1, 'Federalist': 79, 'Jackson

In [17]:
df = pd.DataFrame(party_partisanship)
df.to_csv('party-partisanship.csv', index=False)

In [ ]:
# MAKE PARTY MEMBERSHIP COUNTS OF MAJOR PARTIES

leg = pd.read_csv('legislators.csv')
part = pd.read_csv('partisans.csv')


cong = {}

for i in range(1,114):
    cong[i] = []
    for index, row in leg.iterrows():        
        if row['c'+str(i)] == 1:
            cong[i].append((row.vote_id, row.party))

byparty = {}

for key in cong.keys():
    for person in cong[key]:
        party = person[1]
        if party in byparty:
            byparty[party][key] += 1
        else:
            byparty[party] = [0]*114

print byparty

for key in byparty.keys():
    if key not in part.columns:
        byparty.pop(key, None)

print byparty

df = pd.DataFrame(byparty)
df.to_csv('party_membership.csv', index=False)